In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
from keras.layers import LeakyReLU
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, AveragePooling2D, Activation, Reshape, Concatenate
from keras.layers.experimental.preprocessing import RandomContrast # random contrast change

from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import dlib
import cv2
from PIL import Image
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
import os

In [ ]:
Face Detection:

In [ ]:
def detect_face(image):
    face_detector = dlib.get_frontal_face_detector()
    dets = face_detector(image, 1)

    result = []
    for d in dets:
        xmin, ymin, xmax, ymax = d.left(), d.top(), d.right(), d.bottom()
        xmin = max(xmin, 0)
        xmax = max(xmax, 0)
        ymin = max(ymin, 0)
        ymax = max(ymax, 0)
        result.append([xmin, xmax, ymin, ymax])
    return result

Filter data set use face detection:

In [ ]:
def filter_data_set(filePath, save_path):
    file_lst = os.listdir(filePath)
    print(file_lst)
    for file in file_lst:
        image_name = file[:-4]
        test_image = cv2.imread(filePath + file)
        result = detect_face(test_image)
        num_face = len(result)
        print("Number of face: %d" % (num_face))
        if num_face == 1:
            xmin, xmax, ymin, ymax = result[0]
            # test_image = cv2.rectangle(test_image, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
            cropped = test_image[ymin:ymax, xmin:xmax]
            img = Image.fromarray(cropped)
            img = np.array(img.resize(size=(48, 48)))
            cv2.imwrite(save_path + image_name + ".jpg", img)

This will generate the new data set, do not run this, it will take long, you can use "../input/new-train-data/new_train/" and "../input/new-test-data/new_test/" directly.

In [ ]:
expression_lst = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
save_path = 'new_train/surprise/'
for expression in expression_lst:
    filePath = '../input/fer2013/train/' + expression + '/'
    save_path = '../input/new-train-data/new_train/' + expression + '/'
    filter_data_set(filePath, save_path)
    filePath = '../input/fer2013/test/' + expression + '/'
    save_path = '../input/new-test-data/new_test/' + expression + '/'
    filter_data_set(filePath, save_path)

**This is for new data set filtered with face detection.**

In [ ]:
train_data_generator = ImageDataGenerator(rescale=1.0/255.0, zoom_range=0.3, horizontal_flip=True)

train_data = train_data_generator.flow_from_directory("../input/new-train-data/new_train", 
                                                      target_size = (48, 48),
                                                      color_mode = 'grayscale',
                                                      batch_size = 64,
                                                      class_mode = 'categorical',
                                                      shuffle = True,
                                                      seed = 0)

test_data_generator = ImageDataGenerator(rescale=1.0/255.0)
test_data = test_data_generator.flow_from_directory("../input/new-test-data/new_test", 
                                                    target_size = (48, 48),
                                                    color_mode = 'grayscale',
                                                    batch_size = 64,
                                                    class_mode = 'categorical',
                                                    shuffle = True, 
                                                    seed = 0)

### SIFT/ORB feature extraction for Hybrid CNN (Nizar)

In [ ]:
os.mkdir('/kaggle/working/orb_train/')
os.mkdir('/kaggle/working/orb_test/')
expressions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
for expression in expressions:
    save_path = '/kaggle/working/orb_train/' + expression
    os.mkdir(save_path)
    save_path = '/kaggle/working/orb_test/' + expression
    os.mkdir(save_path)

In [ ]:
orb=cv2.ORB_create()
orb.setEdgeThreshold(5)
def compute_ORB(img):
    kp = orb.detect(img, None)
    kp, des = orb.compute(img, kp)
    return des

def ORB_BoF(k):
    """ 
    k represents number of clusters for bag of features. 
    """
    expressions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    visual_dict = []
    none_count = 0
    for expression in expressions:
        print(expression)
        dir_name = '../input/new-train-data/new_train/' + expression + '/'
        for fname in os.listdir(dir_name):
            fpath = os.path.join(dir_name, fname)
            img = cv2.imread(fpath, 0)
            des = compute_ORB(img)
            if des is not None:
                visual_dict.append(des)
            else:
                print("Found none.")
                none_count += 1
    print(f"Nones: {none_count}")
    visual_dict_vector = np.vstack(visual_dict)
    print(visual_dict_vector.shape)
    kmeans = MiniBatchKMeans(n_clusters=k, batch_size=128).fit(visual_dict_vector)
    train_BoF = np.zeros((len(visual_dict), k))
    n = 0
    for expression in expressions:
        print(expression)
        dir_name = '../input/new-train-data/new_train/' + expression + '/'
        save_path = '/kaggle/working/orb_train/' + expression +'/'
        for fname in os.listdir(dir_name):
            fpath = os.path.join(save_path, fname[:-4]+'.npy')
            cluster_label = kmeans.predict(visual_dict[n])
            feature_histogram = np.array([(cluster_label==i).mean() for i in range(k)])
            np.save(fpath, feature_histogram) # .. save feature_histogram to file
            train_BoF[n] = feature_histogram
            n += 1
    # do same for test set
    test_BoF = []
    for expression in expressions:
        print(expression)
        dir_name = '../input/new-test-data/new_test/' + expression + '/'
        save_path = '/kaggle/working/orb_test/' + expression +'/'
        for fname in os.listdir(dir_name):
            fpath = os.path.join(dir_name, fname)
            img = cv2.imread(fpath)
            des = compute_ORB(img)
            cluster_label = kmeans.predict(des)
            feature_histogram = np.array([(cluster_label==i).mean() for i in range(k)])
            np.save(save_path+fname[:-4]+'.npy', feature_histogram) # .. save feature_histogram to file
            test_BoF.append(feature_histogram)
    test_BoF = np.vstack(test_BoF); print(test_BoF.shape)
    return train_BoF, test_BoF
train_BoF, test_BoF = ORB_BoF(100)

In [ ]:
expressions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
def get_labels():
    train_labels = []
    for i, expression in enumerate(expressions):
        dir_name = '../input/new-train-data/new_train/' + expression + '/'
        label = np.zeros((len(os.listdir(dir_name))))+i
        train_labels.append(label)
    train_labels = np.concatenate(train_labels)
    test_labels = []
    for i, expression in enumerate(expressions):
        dir_name = '../input/new-test-data/new_test/' + expression + '/'
        label = np.zeros((len(os.listdir(dir_name))))+i
        test_labels.append(label)
    test_labels = np.concatenate(test_labels)
    return train_labels, test_labels

def one_hot(labels):
    one_hot_vector = np.zeros((labels.shape[0], 7))
    for i in range(labels.shape[0]):
        one_hot_vector[i, int(labels[i])] = 1.
    return one_hot_vector

y_train, y_test = get_labels()
Y_train, Y_test = one_hot(y_train), one_hot(y_test)
Y_train.shape

# CNN-ORB Hybrid

Here we build the generator for the hybrid model, since it requires two inputs we need to make a custom data generator that takes both inputs.

In [ ]:
train_orb_generator = ImageDataGenerator()
train_BoF = train_BoF.reshape(train_BoF.shape[0], train_BoF.shape[1],1,1)
train_orb = train_orb_generator.flow(train_BoF, 
                                     Y_train,
                                      batch_size = 64,
                                      shuffle = True, 
                                      seed=0)

test_orb_generator = ImageDataGenerator()
test_BoF = test_BoF.reshape(test_BoF.shape[0], test_BoF.shape[1],1,1)
test_orb = test_orb_generator.flow(test_BoF, 
                                   Y_test,
                                    batch_size = 64,
                                    shuffle = True, 
                                    seed=0)

# adapted from: https://stackoverflow.com/questions/49404993/keras-how-to-use-fit-generator-with-multiple-inputs 
class JoinedGen(tf.keras.utils.Sequence):
    def __init__(self, input_gen1, input_gen2):
        self.gen1 = input_gen1
        self.gen2 = input_gen2
        print(len(input_gen1),len(input_gen2))
        assert len(input_gen1) == len(input_gen2)

    def __len__(self):
        return self.gen1.__len__()

    def __getitem__(self, index):
        X1_batch, Y_batch = self.gen1.__getitem__(index)
        X2_batch, Y_batch = self.gen2.__getitem__(index)
        X_batch = [X1_batch, X2_batch]
        return X_batch, Y_batch

    def on_epoch_end(self):
        self.gen1.on_epoch_end()
        self.gen2.on_epoch_end()
        self.gen2.index_array = self.gen1.index_array
train_gen = JoinedGen(train_data, train_orb)
test_gen = JoinedGen(test_data, test_orb)

In [ ]:
# model A: CNN with raw image input
in1 = keras.Input(shape=(48, 48, 1), name="img")
x = Conv2D(128, kernel_size=(3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1))(in1)
x = Conv2D(128, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2, 2)(x)
x = Dropout(0.1)(x)
x = Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
x = Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1))(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.1)(x)
x = Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1))(x)
x = Conv2D(512, kernel_size=(3, 3), activation= tf.keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(2, 2)(x)
x = Dropout(0.1)(x)
out1 = Flatten()(x)

### Model B: ORB input
in2 = keras.Input(shape=(100, 1, 1), name="orb")
intermediate = Reshape((100,))(in2)
out2 = Dense(4096, activation=tf.keras.layers.LeakyReLU(alpha=0.1))(intermediate)
out2 = Dropout(0.5)(out2)

# Merged model
x = Concatenate()([out1, out2])
out3 = Dense(2048, activation=tf.keras.layers.LeakyReLU(alpha=0.1))(x)
out3 = Dropout(0.5)(out3)
out3 = Dense(7, activation='softmax')(out3)

# final model: ORB-CNN hybrid
model = keras.Model([in1, in2], out3, name='ORB-CNN')
model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
          loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()
    

In [ ]:
plot_model(model, to_file='model_visualize.png', show_shapes=True, show_layer_names=True)

In [ ]:
check_point_loss = tf.keras.callbacks.ModelCheckpoint(filepath='model_loss.h5',
                                                      save_best_only=True,
                                                      verbose=1,
                                                      mode='min',
                                                      moniter='val_loss')

check_point_acc = tf.keras.callbacks.ModelCheckpoint(filepath='model_acc.h5',
                                                      save_weights_only=True,
                                                      verbose=1,
                                                      mode='max',
                                                      moniter='val_accuracy')

# Reduce learning rate when a metric has stopped improving.
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            factor=0.2, 
                                                            patience=5, 
                                                            verbose=1, 
                                                            min_delta=0.0001)


logs = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1),
csv_logger = tf.keras.callbacks.CSVLogger('training.log')

callbacks = [check_point_loss,check_point_acc, reduce_learning_rate, csv_logger]

history = model.fit(train_gen,
                    steps_per_epoch=train_data.n // 64,
                    epochs=20,
                    validation_data=test_gen,
                    validation_steps=test_data.n // 64,
                    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
plt.title("ORB-CNN Hybrid Model Accuracy")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train', 'test'])
fig=plt.figure()
plt.title("ORB-CNN Hybrid Model Loss")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'])

End of CNN-ORB Hybrid

In [ ]:
train_data_generator = ImageDataGenerator(rescale=1.0/255.0, zoom_range=0.3, horizontal_flip=True)

train_data = train_data_generator.flow_from_directory("../input/fer2013/train/", 
                                                      target_size = (48, 48),
                                                      color_mode = 'grayscale',
                                                      batch_size = 64,
                                                      class_mode = 'categorical',
                                                      shuffle = True)

test_data_generator = ImageDataGenerator(rescale=1.0/255.0)
test_data = test_data_generator.flow_from_directory("../input/fer2013/test/", 
                                                    target_size = (48, 48),
                                                    color_mode = 'grayscale',
                                                    batch_size = 64,
                                                    class_mode = 'categorical',
                                                    shuffle = True)

CNN model:


In [ ]:
model = tf.keras.models.Sequential()   

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape=(48,48,1)))
model.add(Conv2D(128, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.1)) #try 0.15

model.add(Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding='same'))
model.add(Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape=(48,48,1)))
model.add(Conv2D(512, kernel_size=(3, 3), activation= tf.keras.layers.LeakyReLU(alpha=0.1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.1))


model.add(Flatten())

model.add(Dense(512,kernel_regularizer=regularizers.l2(0.01),activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
#above we get out 
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
          loss="categorical_crossentropy", metrics=['accuracy'])

model.summary()

Plot CNN model:

In [ ]:
plot_model(model, to_file='model_visualize.png', show_shapes=True, show_layer_names=True)

In [ ]:
check_point_loss = tf.keras.callbacks.ModelCheckpoint(filepath='model_loss.h5',
                                                      save_best_only=True,
                                                      verbose=1,
                                                      mode='min',
                                                      moniter='val_loss')

check_point_acc = tf.keras.callbacks.ModelCheckpoint(filepath='model_acc.h5',
                                                      save_weights_only=True,
                                                      verbose=1,
                                                      mode='max',
                                                      moniter='val_accuracy')

# Reduce learning rate when a metric has stopped improving.
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            factor=0.2, 
                                                            patience=5, 
                                                            verbose=1, 
                                                            min_delta=0.0001)


logs = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1),
csv_logger = tf.keras.callbacks.CSVLogger('training.log')

callbacks = [check_point_loss,check_point_acc, reduce_learning_rate, csv_logger]

history = model.fit(train_data,
                    steps_per_epoch=train_data.n // 64,
                    epochs=40,
                    validation_data=test_data,
                    validation_steps=test_data.n // 64,
                    callbacks=callbacks)

Plot accuracy and loss for CNN model:

In [ ]:
plt.title("CNN Model Accuracy")
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train', 'test'])
fig=plt.figure()
plt.title("CNN Model Loss")
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'test'])

Face detection

You need to change the image_name and the image path for running different images.

In [ ]:
image_name = "test3"
test_image = cv2.imread('../input/face-detection/test3.jpeg', 0)
h, w = test_image.shape[0], test_image.shape[1]
print(test_image.shape)
result = detect_face(test_image)
num_face = len(result)
print("Number of face: %d" % (num_face))
# num_face
for i in range(num_face):
    xmin, xmax, ymin, ymax = result[i]
    xmin = min(xmin, w)
    xmax = min(xmax, w)
    ymin = min(ymin, h)
    ymax = min(ymax, h)
    test_image = cv2.rectangle(test_image, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
    cropped = test_image[ymin:ymax, xmin:xmax]
    img = Image.fromarray(cropped)
    img = img.resize(size=(48, 48))
    plt.imshow(img, cmap='gray')
    img_input = image.img_to_array(img)
    img_input = np.expand_dims(img_input, axis=0)
    img_input /= 255
    emotion_pribability = model.predict(img_input)
    print(emotion_pribability)
    emotion_pred = np.argmax(emotion_pribability, axis=1)
    print(emotion_pred)
    class_labels = test_data.class_indices
    print(class_labels)
    emotion = [k for k, v in class_labels.items() if v == emotion_pred]
    print(emotion[0])
    text_x = xmin
    text_y = int(ymin + (ymax - ymin) / 2)
    cv2.putText(test_image, emotion[0], (text_x, text_y), 1, 1, (0, 255, 0), 1)
plt.imshow(test_image, cmap='gray')